# Amazon Bedrock Knowledge Bases

이 노트북은 실습을 위해 Amazon Bedrock Knowledge Bases를 생성하고, 데이터 소스에 해당하는 Amazon S3 버킷을 생성합니다.

#### 노트북 워크스루

현재 Git Repository의 dataset 디렉토리에 저장된 데이터를 Amazon S3 버킷에 업로드하고, AWS 콘솔에서 UI를 활용해 Amazon Bedrock Knowledge bases를 생성합니다. 

## 설정
나머지 셀을 실행하기 전에 아래 셀을 먼저 실행해 필요한 라이브러리를 설치하고 Bedrock에 연결하세요.

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [ ]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

### Amazon S3에 데이터 업로드
벡터 데이터베이스에 벡터화 해서 저장할 문서 파일을 업로드 합니다. 

In [ ]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
print(boto3.__version__)

In [ ]:
def create_bucket_if_not_exists(bucket_name):
    """S3 버킷이 존재하지 않으면 생성합니다."""
    try:
        s3_client.head_bucket(Bucket=bucket_name)
        print(f"Bucket {bucket_name} already exists")
    except:
        print(f"Creating bucket {bucket_name}...")
        try:
            if region == 'us-east-1':
                s3_client.create_bucket(Bucket=bucket_name)
            else:
                s3_client.create_bucket(
                    Bucket=bucket_name,
                    CreateBucketConfiguration={'LocationConstraint': region}
                )
            print(f"✓ Bucket {bucket_name} created successfully")
        except Exception as e:
            print(f"Error creating bucket: {e}")
            raise

def upload_directory(path, bucket_name):
        # 버킷이 없으면 생성
        create_bucket_if_not_exists(bucket_name)
        
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

data_bucket_name = f'bedrock-kb-{suffix}-1'
upload_directory("../dataset", data_bucket_name)

### Amazon Knowledge base 생성

1. AWS Console에서 Amazon Bedorck을 검색하여 Amazon Bedrock Console로 이동합니다.
![](images/rag_1.png)

2. 왼쪽 사이드바의 Knowledge bases를 선택한 후 Create 버튼의 [Knowledge Base with vector store]를 선택합니다. 
![](images/rag_2.png)

3. Knowledge Base name에 `bedrock-sample-knowledge-base`를 지정하고, IAM permission은 [Create and use a new service role]을 선택하여 필요한 기본 권한이 포함된 IAM Role을 새로 생성합니다. data source는 Amazon S3를 선택합니다.
![](images/rag_3.png)

4. Data source 설정에서 Data source name은 기본값을 그대로 사용합니다. 그리고 문서가 저장된 S3 버킷을 지정하기 위해 S3 URI 항목의 Browse S3 버튼을 선택합니다.
![](images/rag_4.png)

5. 앞서 문서를 업로드한 `bedrock-kb-`로 시작하는 Amazon S3 Bucket을 선택합니다.
![](images/rag_5.png)

6. Parsing strategy는 기본값인 `Amazon Bedrock default parser`를 사용합니다. 이 Parser는 텍스트만 추출할 수 있기 때문에 이미지와 같은 멀티모달 컨텐츠들은 무시하고 가져옵니다.
![](images/rag_6.png)

7. Chunking strategy는 [Fixed-size chunking]을 선택해서 기본으로 지정된 최대 300 토큰수 이내로 청크를 나누도록 합니다. 이 때 청크의 20% 크기 정도는 overlap 되도록 설정합니다.
![](images/rag_7.png)

8. 데이터를 벡터화 시킬 때 사용할 임베딩 모델을 선택합니다. 임베딩 모델로는 Amazon Titan Text Embeddings V2 모델을 사용합니다.
![](images/rag_8.png)
 
9. 벡터 데이터베이스는 신규로 생성하기 위해 [Quick create a new vector store]를 선택하고, Vector store type으로는 [Amazon OpenSearch Serverless]를 선택합니다.
![](images/rag_9.png)

10. 전체 설정들을 리뷰한 후 [Create Knowledge Base] 버튼을 선택해서 Amazon Bedrock Knowledge Base를 생성합니다.
![](images/rag_10.png)

생성까지 약 10분 정도 소요 됩니다. 

### Data Source의 데이터를 벡터 데이터베이스에 동기화

1. Amazon Bedrock KB가 생성된 후 Data source의 항목을 선택한 후 Sync 버튼을 선택합니다. 이 작업은 지정된 S3 Bucket에 포함된 데이터 수나 크기에 따라 소요시간이 달라질 수 있습니다. 실습에서는 PDF 문서 파일 하나만 사용하기 때문에 2~3분 정도 소요됩니다.
![](images/rag_11.png)

2. Sync가 완료되면 [Status]가 `Available`로 전환되고, 상단의 Test [Knowledge Base] 버튼이 활성화 됩니다. [Test Knowledge Base] 버튼을 선택합니다. 
![](images/rag_12.png)

3. [Retrieval and response generation] 항목에서 [Retrieval and response generation]을 선택하고 Model은 [Claude Sonnet 4.5]를 선택합니다. 
![](images/rag_13.png)

4. 오른쪽 테스트 채팅 창에서 `AWS의 Security 서비스들에는 어떤 것들이 있습니까?`를 입력해봅니다.
![](images/rag_14.png)

5. Detail 버튼을 클릭해보면 생성된 답변이 어떻게 도출되었는지 상세 내용을 확인할 수 있습니다. 벡터데이터베이스에 저장된 청크 정보와 해당 청크가 어떤 파일의 몇 페이지에 있었는지에 대한 메타데이터를 확인할 수 있습니다. 
![](images/rag_15.png)

6. [Retrieval only]로도 선택해서 AWS 관련한 자유로운 질문을 해보세요. (예시 - ElastiCache에 대해 설명해주세요.)

### 다읍 실습을 위한 준비

Amazon Bedrock Knowledge Bases 페이지에서 앞서 생성한 Knowledge Base를 선택한 후 아래 화면의 Knowledge Base ID를 복사합니다.
![](images/rag_16.png)

In [ ]:
# kb_id = "여기에 Knowledge Base ID를 입력해주세요"
kb_id = "LMWDDN0YQA"
%store kb_id